# Load Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir('../../input'))

['sample_submission.csv', 'test_identity.csv', 'test_transaction.csv', 'train_identity.csv', 'train_transaction.csv']


# 1. Prepare Data

## 1.1 Load Datasets

In [2]:
df_train_t = pd.read_csv('../../input/train_transaction.csv', index_col='TransactionID')
df_train_i = pd.read_csv('../../input/train_identity.csv', index_col='TransactionID')
df_test_t = pd.read_csv('../../input/test_transaction.csv', index_col='TransactionID')
df_test_i = pd.read_csv('../../input/test_identity.csv', index_col='TransactionID')
sample_submission = pd.read_csv('../../input/sample_submission.csv')

print('train_transaction의 shape : ', df_train_t.shape)
print('train_identity의 shape : ', df_train_i.shape)
print('test_transaction의 shape : ', df_test_t.shape)
print('test_transaction의 shape : ', df_test_i.shape)

train_transaction의 shape :  (590540, 393)
train_identity의 shape :  (144233, 40)
test_transaction의 shape :  (506691, 392)
test_transaction의 shape :  (141907, 40)


## 1.2 Merge Datasets

In [3]:
df_train = df_train_t.merge(df_train_i, how='left', left_index=True, right_index=True)
df_test = df_test_t.merge(df_test_i, how='left', left_index=True, right_index=True)

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float64(399), int64(3), object(31)
memory usage: 1.9+ GB


## 1.3 Divide Features by Categorical and Numerical

In [5]:
# cat_col_t : categorical columns in transaction dataset
cat_col_t = ['ProductCD','addr1','addr2', 'P_emaildomain', 'R_emaildomain', 'TransactionDT']
cat_col_t.extend(['card' + str(x) for x in range(1,7)]) # card1 ~ card6
cat_col_t.extend(['M' + str(x) for x in range(1,10)]) # M1 ~ M9

# cat_col_i : categorical columns in identity dataset
cat_col_i = ['DeviceType', 'DeviceInfo']
cat_col_i.extend(['id_' + str(x) for x in range(12, 39)]) # id_12 ~ id_38

# cat_col : categorical columns in transaction and identity dataset
cat_col = [*cat_col_t, *cat_col_i]

In [6]:
# num_col_t : numerical columns in transaction dataset
num_col_t = [col for col in list(df_train_t.columns) if col not in [*cat_col_t, 'isFraud']]

# num_col_i : numerical columns in identity dataset
num_col_i = [col for col in list(df_train_i.columns) if col not in cat_col_i]

# num_col : numerical columns in transaction and identity dataset
num_col =[*num_col_t, *num_col_i]

In [7]:
print("number of transaction dataset's columns : ", df_train_t.shape[1])
print("cat_col_t's length : ", len(cat_col_t))
print("num_col_t's length : ", len(num_col_t))
print("sum of boths : ", len(cat_col_t) + len(num_col_t))

number of transaction dataset's columns :  393
cat_col_t's length :  21
num_col_t's length :  371
sum of boths :  392


In [8]:
print("number of transaction dataset's columns : ", df_train_i.shape[1])
print("cat_col_i's length : ", len(cat_col_i))
print("num_col_i's length : ", len(num_col_i))
print("sum of boths : ", len(cat_col_i) + len(num_col_i))

number of transaction dataset's columns :  40
cat_col_i's length :  29
num_col_i's length :  11
sum of boths :  40


# 2. Summary of Features

In [3]:
def make_cat_summary(df, features) :

    summary = df[features].dtypes.reset_index()
    summary.rename(columns={'index': 'Name', 0: 'Dtype'}, inplace=True)
    summary['DataSets'] = ['transaction' if col in cat_col_t else 'identity' for col in features]
    summary['NullCnt'] = [df[col].isnull().sum() for col in features]
    summary['NullRt'] = [np.round((df[col].isnull().sum())/df.shape[0], 2) for col in features]
    summary['UniqueCnt'] = [df[col].nunique() for col in features]
    Values = []
    for col in features :
        if df[col].nunique() <= 5 :
            val = list(df[col].value_counts().reset_index()['index'])
            val.sort()
            Values.append(', '.join(str(v) for v in val))
        else :
            Values.append('-')    
    summary['Values'] = Values
    summary['MinValue'] = [df.loc[df[col].notnull(), col].min() for col in features]
    summary['MaxValue'] = [df.loc[df[col].notnull(), col].max() for col in features]
    
    return summary

In [4]:
cat_summary = make_cat_summary(df_train, cat_col)
cat_summary

NameError: name 'df_train' is not defined

In [11]:
def make_num_summary(df, features) :
    
    summary = df[features].dtypes.reset_index()
    summary.rename(columns={'index': 'Name', 0: 'Dtype'}, inplace=True)
    summary['DataSets'] = ['transaction' if col in num_col_t else 'identity' for col in features]
    summary['NullCnt'] = [df[col].isnull().sum() for col in features]
    summary['NullRt'] = [np.round((df[col].isnull().sum())/df.shape[0], 2) for col in features]
    summary['MinValue'] = [df.loc[df[col].notnull(), col].min() for col in features]
    summary['Q25'] = [df[col].quantile([0.25]).values[0] for col in features]
    summary['Q50'] = [df[col].quantile([0.50]).values[0] for col in features]
    summary['Q75'] = [df[col].quantile([0.75]).values[0] for col in features]
    summary['MaxValue'] = [df.loc[df[col].notnull(), col].max() for col in features]
    summary['Mean'] = [df.loc[df[col].notnull(), col].mean() for col in features]
    summary['Std'] = [df.loc[df[col].notnull(), col].std() for col in features]

    return summary

In [12]:
num_summary = make_num_summary(df_train, num_col)
num_summary

,Name,Dtype,DataSets,NullCnt,NullRt,MinValue,Q25,Q50,Q75,MaxValue,Mean,Std
0,TransactionAmt,float64,transaction,0,0.00,0.251,43.321000,68.769000,125.000000,31937.391000,135.027176,239.162522
1,dist1,float64,transaction,352271,0.60,0.000,3.000000,8.000000,24.000000,10286.000000,118.502180,371.872026
2,dist2,float64,transaction,552913,0.94,0.000,7.000000,37.000000,206.000000,11623.000000,231.855423,529.053494
3,C1,float64,transaction,0,0.00,0.000,1.000000,1.000000,3.000000,4685.000000,14.092458,133.569018
4,C2,float64,transaction,0,0.00,0.000,1.000000,1.000000,3.000000,5691.000000,15.269734,154.668899
5,C3,float64,transaction,0,0.00,0.000,0.000000,0.000000,0.000000,26.000000,0.005644,0.150536
6,C4,float64,transaction,0,0.00,0.000,0.000000,0.000000,0.000000,2253.000000,4.092185,68.848459
7,C5,float64,transaction,0,0.00,0.000,0.000000,0.000000,1.000000,349.000000,5.571526,25.786976
8,C6,float64,transaction,0,0.00,0.000,1.000000,1.000000,2.000000,2253.000000,9.071082,71.508467
9,C7,float64,transaction,0,0.00,0.000,0.000000,0.000000,0.000000,2255.000000,2.848478,61.727304


# 3. Feature Engineering

In [13]:
def value_acc_freq(df, col) :
        
    df_vc = df[col].value_counts().reset_index()
    df_vc.rename(columns={'index': 'value', col: 'cnt'}, inplace=True)
    df_vc['accCntRt'] = df_vc['cnt'].cumsum() / len(df[df[col].notnull()])
    
    return df_vc

In [14]:
def replace_to_others(df, col, rate=None, cnt=None) :
    
    df_vc = df[col].value_counts().reset_index()
    df_vc.rename(columns={'index': 'value', col: 'cnt'}, inplace=True)
    df_vc['accCntRt'] = df_vc['cnt'].cumsum() / len(df[df[col].notnull()])
    target_list = []
    
    if rate != None :
        target_list = list(df_vc[df_vc['accCntRt'] >= float(rate)]['value'])
    if cnt != None :
        target_list = list(df_vc[df_vc['cnt'] < int(10)]['value'])
        
    dataType = str(df_vc['value'].values.dtype)
    replace_value = 'OTHERS'
    if dataType.find('int') == 0 :
        replace_value = int(99999)
    elif dataType.find('float') == 0 :
        replace_value = float(99999)
    
    df.loc[df[col].isin(target_list), col] = replace_value

## 3.1 Categorical Features

In [15]:
def cat_feature_engineering(df) :
    # addr1
    replace_to_others(df, 'addr1', rate=0.95)

    # addr2
    replace_to_others(df, 'addr2', cnt=10)

    # P_emaildomain
    df.loc[df['P_emaildomain'].str.contains('gmail', na=False), 'P_emaildomain'] = 'GMAIL'
    df.loc[df['P_emaildomain'].str.contains('yahoo', na=False), 'P_emaildomain'] = 'YAHOO'
    df.loc[df['P_emaildomain'].str.contains('hotmail', na=False), 'P_emaildomain'] = 'HOTMAIL'
    df.loc[df['P_emaildomain'].str.contains('live', na=False), 'P_emaildomain'] = 'LIVE'
    df.loc[df['P_emaildomain'].str.contains('netzero', na=False), 'P_emaildomain'] = 'NETZERO'
    df.loc[df['P_emaildomain'].str.contains('outlook', na=False), 'P_emaildomain'] = 'OUTLOOK'
    replace_to_others(df, 'P_emaildomain', cnt=250)

    # R_emaildomain
    df.loc[df['R_emaildomain'].str.contains('gmail', na=False), 'R_emaildomain'] = 'GMAIL'
    df.loc[df['R_emaildomain'].str.contains('yahoo', na=False), 'R_emaildomain'] = 'YAHOO'
    df.loc[df['R_emaildomain'].str.contains('hotmail', na=False), 'R_emaildomain'] = 'HOTMAIL'
    df.loc[df['R_emaildomain'].str.contains('live', na=False), 'R_emaildomain'] = 'LIVE'
    df.loc[df['R_emaildomain'].str.contains('netzero', na=False), 'R_emaildomain'] = 'NETZERO'
    df.loc[df['R_emaildomain'].str.contains('outlook', na=False), 'R_emaildomain'] = 'OUTLOOK'
    replace_to_others(df, 'R_emaildomain', cnt=80)

    # TransactionDT
    # Reference : https://www.kaggle.com/shkim4738/extensive-eda-and-modeling-xgb-hyperopt
    import datetime

    START_DATE = '2017-12-01'
    startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
    df['Date'] = df['TransactionDT'].apply(lambda x : (startdate + datetime.timedelta(seconds=x)))

    df['Weekdays'] = df['Date'].dt.dayofweek
    df['Hours'] = df['Date'].dt.hour
    df['Days'] = df['Date'].dt.day

    df.drop(['Date'], axis=1, inplace=True)

    # DeviceInfo
    df.loc[df['DeviceInfo'].str.contains('SAMSUNG', na=False), 'DeviceInfo'] = 'SAMSUNG'
    df.loc[df['DeviceInfo'].str.contains('SM', na=False), 'DeviceInfo'] = 'SM'
    df.loc[df['DeviceInfo'].str.contains('rv', na=False), 'DeviceInfo'] = 'RV'
    df.loc[df['DeviceInfo'].str.contains('Moto', na=False), 'DeviceInfo'] = 'MOTO'
    df.loc[df['DeviceInfo'].str.contains('HUAWEI', na=False), 'DeviceInfo'] = 'HUAWEI'
    df.loc[df['DeviceInfo'].str.contains('Huawei', na=False), 'DeviceInfo'] = 'HUAWEI'
    df.loc[df['DeviceInfo'].str.contains('LG-', na=False), 'DeviceInfo'] = 'LG'
    df.loc[df['DeviceInfo'].str.contains('Android', na=False), 'DeviceInfo'] = 'ANDROID'
    df.loc[df['DeviceInfo'].str.contains('Linux', na=False), 'DeviceInfo'] = 'LINUX'
    df.loc[df['DeviceInfo'].str.contains('HTC', na=False), 'DeviceInfo'] = 'HTC'
    df.loc[df['DeviceInfo'].str.contains('Hisense', na=False), 'DeviceInfo'] = 'HISENSE'
    df.loc[df['DeviceInfo'].str.contains('Blade', na=False), 'DeviceInfo'] = 'BLADE'
    df.loc[df['DeviceInfo'].str.contains('BLADE', na=False), 'DeviceInfo'] = 'BLADE'
    df.loc[df['DeviceInfo'].str.contains('ASUS', na=False), 'DeviceInfo'] = 'ASUS'
    df.loc[df['DeviceInfo'].str.contains('Redmi', na=False), 'DeviceInfo'] = 'REDMI'
    df.loc[df['DeviceInfo'].str.contains('iOS', na=False), 'DeviceInfo'] = 'iOS'
    df.loc[df['DeviceInfo'].str.contains('MacOS', na=False), 'DeviceInfo'] = 'MacOS'
    device = ['SAMSUNG','SM','RV','MOTO','HUAWEI','LG','ANDROID','LINUX','HTC','HISENSE','BLADE','ASUS','REDMI',
              'Windows','iOS', 'MacOS', 'Trident/7.0']
    df.loc[(~df['DeviceInfo'].isin(device)) & (df['DeviceInfo'].notnull()), 'DeviceInfo'] = 'OTHERS'

    # id_30
    df.loc[df['id_30'].str.contains('Windows', na=False), 'id_30'] = 'WINDOWS'
    df.loc[df['id_30'].str.contains('iOS', na=False), 'id_30'] = 'iOS'
    df.loc[df['id_30'].str.contains('Mac OS X', na=False), 'id_30'] = 'MacOS'
    df.loc[df['id_30'].str.contains('Android', na=False), 'id_30'] = 'ANDROID'

    # id_31
    df.loc[df['id_31'].str.contains('chrome', na=False), 'id_31'] = 'CHROME'
    df.loc[df['id_31'].str.contains('firefox', na=False), 'id_31'] = 'FIREFOX'
    df.loc[df['id_31'].str.contains('edge', na=False), 'id_31'] = 'EDGE'
    df.loc[df['id_31'].str.contains('ie ', na=False), 'id_31'] = 'IE'
    df.loc[df['id_31'].str.contains('safari', na=False), 'id_31'] = 'SAFARI'
    df.loc[df['id_31'].str.contains('opera', na=False), 'id_31'] = 'OPERA'
    df.loc[df['id_31'].str.contains('samsung', na=False), 'id_31'] = 'SAMSUNG'
    df.loc[df['id_31'].str.contains('Samsung', na=False), 'id_31'] = 'SAMSUNG'
    df.loc[df['id_31'].str.contains('android', na=False), 'id_31'] = 'ANDROID'
    df.loc[df['id_31'].str.contains('Android', na=False), 'id_31'] = 'ANDROID'
    device2 = ['CHROME','FIREFOX','EDGE','IE','SAFARI','OPERA','SAMSUNG','ANDROID']
    df.loc[(~df['id_31'].isin(device2)) & (df['id_31'].notnull()), 'id_31'] = 'OTHERS'

    # id_33
    replace_to_others(df, 'id_33', cnt=10)

In [16]:
cat_feature_engineering(df_train)
cat_feature_engineering(df_test)

In [17]:
cat_col_t.extend(['Weekdays','Hours','Days'])
cat_col = [*cat_col_t, *cat_col_i]

## 3.2 Numerical Features

생략

## 3.3 Missing Values

In [18]:
cat_col_null_over90 = list(cat_summary.loc[cat_summary['NullRt'] >= float(0.9), 'Name'].values)
num_col_null_over90 = list(num_summary.loc[num_summary['NullRt'] >= float(0.9), 'Name'].values)
col_null_over90 = [*cat_col_null_over90, *num_col_null_over90]
col_null_over90

['id_18',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'dist2',
 'D7',
 'D13',
 'id_07',
 'id_08']

In [19]:
df_train = df_train.drop(col_null_over90, axis=1)
df_test = df_test.drop(col_null_over90, axis=1)

In [20]:
cat_col = [col for col in cat_col if col not in cat_col_null_over90]
num_col = [col for col in num_col if col not in num_col_null_over90]

`sklearn`의 분류기들은 Null 값을 입력인자로 받지 못한다.

In [21]:
for col in cat_col :
    if df_train[col].isnull().sum() != 0 :
        df_train[col].fillna(value='NaN', inplace=True) # NaN 범주로 결측값 처리

In [22]:
for col in num_col :
    if df_train[col].isnull().sum() != 0 :
        df_train[col].fillna(df_train[col].mean(), inplace=True) # 평균값으로 결측치 처리

# 4. Encoding

In [23]:
for col in cat_col :
    if col in df_train.columns :
        le = LabelEncoder()
        le.fit(list(df_train[col].astype(str).values) + list(df_test[col].astype(str).values))
        df_train[col] = le.transform(list(df_train[col].astype(str).values))
        df_test[col] = le.transform(list(df_test[col].astype(str).values))

# 5. Set X, y

In [24]:
y_train = df_train['isFraud']
X_train = df_train.drop('isFraud', axis=1)
X_test = df_test

## 5.1 Make Validation set

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)

# 6. Modeling

In [26]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_valid)
print("DecisionTreeClassifier's roc_auc_score : {0:.4f}".format(roc_auc_score(y_valid, dt_pred)))

DecisionTreeClassifier's roc_auc_score : 0.7884


In [27]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_valid)
print("RandomForestClassifier's roc_auc_score : {0:.4f}".format(roc_auc_score(y_valid, rf_pred)))

RandomForestClassifier's roc_auc_score : 0.7142


In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_valid)
print("LogisticRegression's roc_auc_score : {0:.4f}".format(roc_auc_score(y_valid, lr_pred)))